<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [3]:
import pathlib
import pandas as pd

In [8]:
layers = ['IT-L23', 'IT-L4', 'IT-L5', 'IT-L6']
bigwig_dir = '/gale/netapp/home/hanliu/ddn/hanliu/mouse_rostral_brain/MultiLevelCluster/BW/MajorType'
bigwig_list = [f'{bigwig_dir}/MajorType.{layer}.CGN-Both.rate.bw' for layer in layers]

In [9]:
bed_list = list(pathlib.Path('bed/').absolute().glob('*bed'))

In [24]:
cmds = []
for bed in bed_list:
    for bigwig in bigwig_list:
        layer = bed.name.split('.')[0]
        bg_layer = pathlib.Path(bigwig).name.split('.')[1]
        output_path = pathlib.Path().absolute() / f'{layer}DMR_{bg_layer}bw.tsv.gz'
        cmd = f'computeMatrix reference-point -S {bigwig} -R {bed} --referencePoint center -bs 30 -a 3000 -b 3000 -o {output_path} -p 40'
        cmds.append(cmd)
        
with open('cmds.txt', 'w') as f:
    f.write('\n'.join(cmds))

In [25]:
qsub_str = """
#!/bin/bash
#$ -N bw
#$ -V
#$ -pe smp 1
#$ -l h_vmem=3G
#$ -l h_rt=99:99:99
#$ -l s_rt=99:99:99
#$ -e /gale/netapp/scratch2/hanliu/mouse_rostral_brain/ITSpatial/DMRMatrix/AggLayerDMR/error.log
#$ -o /gale/netapp/scratch2/hanliu/mouse_rostral_brain/ITSpatial/DMRMatrix/AggLayerDMR/output.log

yap qsub --command_file_path /gale/netapp/scratch2/hanliu/mouse_rostral_brain/ITSpatial/DMRMatrix/AggLayerDMR/cmds.txt --working_dir /gale/netapp/scratch2/hanliu/mouse_rostral_brain/ITSpatial/DMRMatrix/AggLayerDMR --project_name qsub --total_cpu 400 --total_mem 2000 --qsub_global_parms "-pe smp=40;-l h_vmem=3G"
"""
with open('qsub.sh', 'w') as f:
    f.write(qsub_str)